In [4]:
%matplotlib inline

In [5]:
import matplotlib.pyplot as plt
import statistics 
from scipy.stats import skew
import numpy as np
import pandas as pd
from google.oauth2 import service_account #Control API Keys
from google.cloud import vision
from pdf2image import convert_from_path #Libary convert pdf file to img file
import os
import cv2
import io
import random
from random import randrange
from IPython.display import Image
from shapely.geometry import Polygon
#import argparse
from enum import Enum
import io
from google.cloud.vision import types
from PIL import Image, ImageDraw
from PIL import ImageFont
from pyresparser import ResumeParser
import shutil
from shutil import copyfile
import random
import string
import spacy
from pandas import ExcelWriter
from termcolor import colored
nlp = spacy.load("en_core_web_sm")

In [24]:
#doc = '/Users/kunal/Documents/VdartResumeProject/VisionAPi/Document_402.pdf'
global docPath
docPath = '/Users/kunal/Documents/VdartResumeProject/VisionAPi/'
global imgTxtVisionAPIPath
imgTxtVisionAPIPath = "/Users/kunal/Documents/VdartResumeProject/APIKEYSGOOGLE/resumeMatcher-pdf2img.json"
global pdfIMGPopplerPath
pdfIMGPopplerPath = '/Users/kunal/Documents/VdartResumeProject/Poppler/poppler-0.68.0_x86/poppler-0.68.0/bin/'
global fontPath
fontPath = '/Users/kunal/Documents/VdartResumeProject/Font/FreeMonoBold.ttf'
global allResumesPath
allResumesPath = "/Users/kunal/Documents/VdartResumeProject/50_resumes/"

In [26]:
#Using API from Google
#Returns a JSON file but text is extracted from it
global keyDIR
keyDIR = imgTxtVisionAPIPath #JSON key file to call the api
credentials = service_account.Credentials.from_service_account_file(keyDIR) #using service account to go through google
global client
client = vision.ImageAnnotatorClient(credentials=credentials) # client api

In [27]:
for i in os.listdir(docPath):
    if i.endswith(".jpg") or i.endswith(".png"):
        print(i)

Document_402_1.jpg
Document_402_1_2.jpg
Document_402_1_2_New.jpg
Document_406_3.jpg


In [28]:
#from other Jupyther notebook
PATHS = [[['/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_1504/Document_1504_1.jpg',
           '/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_1504/Document_1504.pdf'],
          ['/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_1504/Document_1504_2.jpg',
           '/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_1504/Document_1504.pdf']],
         [['/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_502/Document_502_1.jpg',
           '/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_502/Document_502.pdf']],
         [['/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_608/Document_608_1.jpg',
           '/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_608/Document_608.pdf'],
          ['/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_608/Document_608_2.jpg',
           '/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_608/Document_608.pdf'],
          ['/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_608/Document_608_3.jpg',
           '/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_608/Document_608.pdf']],
         [['/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_866/Document_866_1.jpg',
           '/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_866/Document_866.pdf'],
          ['/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_866/Document_866_2.jpg',
           '/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_866/Document_866.pdf']],
         [['/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_882/Document_882_1.jpg',
           '/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_882/Document_882.pdf'],
          ['/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_882/Document_882_2.jpg',
           '/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_882/Document_882.pdf'],
          ['/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_882/Document_882_3.jpg',
           '/Users/kunal/Documents/VdartResumeProject/VisionAPi/Fold_Document_882/Document_882.pdf']]]

In [29]:
ALLPATHSIMG = []
for i in PATHS:
    for j in i:
        ALLPATHSIMG.append(j[0])

In [39]:
%%time
allIMGS = []
for i in ALLPATHSIMG:
    imagePath = i
    runningIMG = RUNALL(imagePath)
    allIMGS.append(runningIMG)
    print("\n\n\n")
    

MaybeErrrorWhenConvertingPolygonToRectangle
MaybeErrrorWhenConvertingPolygonToRectangle
MaybeErrrorWhenConvertingPolygonToRectangle
MaybeErrrorWhenConvertingPolygonToRectangle
MaybeErrrorWhenConvertingPolygonToRectangle
MaybeErrrorWhenConvertingPolygonToRectangle


NameError: name 'croppedTopBottomImgOnly01' is not defined

In [37]:
def findArea(bounds):
    matrix = ((bounds.vertices[0].x, bounds.vertices[0].y),
              (bounds.vertices[1].x, bounds.vertices[1].y),
              (bounds.vertices[2].x, bounds.vertices[2].y),
              (bounds.vertices[3].x, bounds.vertices[3].y))    
    polygon = Polygon(matrix)
    return polygon.area
def detect_Maximum_outlier(data_1):
    #z = (X — μ) / σ
    #Formula for Z score = (Observation — Mean)/Standard Deviation
    outliers=[]
    threshold=3
    mean_1 = np.mean(data_1)
    std_1 =np.std(data_1)
    for y in data_1:
        z_score= (y - mean_1)/std_1 
        if np.abs(z_score) > threshold and y > mean_1:
            outliers.append(y)
    return outliers
def findMaxOutliersIQR(datasetInput):
    dataset = sorted(datasetInput)
    q1, q3= np.percentile(dataset,[25,75])
    iqr = q3 - q1
    upper_bound = q3 +(1.5 * iqr) 
    outliers=[]
    for num in dataset:
        if num>upper_bound:
            outliers.append(num)
    return outliers
def averageOfBothOutliers(data):
    return (min(detect_Maximum_outlier(data)) + min(findMaxOutliersIQR(data)))/2
def convertBoundBoxtodiagonalRectangual(polygon):
    #print(polygon)
    if (abs(polygon.vertices[0].x - polygon.vertices[3].x)<=1 and 
        abs(polygon.vertices[0].y - polygon.vertices[1].y)<=1 and 
        abs(polygon.vertices[1].x - polygon.vertices[2].x)<=1 and 
        abs(polygon.vertices[2].y - polygon.vertices[3].y)<=1):
        matrixCrop = (min(polygon.vertices[0].x, polygon.vertices[3].x),
                      min(polygon.vertices[0].y, polygon.vertices[1].y),
                      max(polygon.vertices[1].x, polygon.vertices[2].x),
                      max(polygon.vertices[2].y, polygon.vertices[3].y))
    else:
        matrixCrop = (min(polygon.vertices[0].x, polygon.vertices[3].x),
                      min(polygon.vertices[0].y, polygon.vertices[1].y),
                      max(polygon.vertices[1].x, polygon.vertices[2].x),
                      max(polygon.vertices[2].y, polygon.vertices[3].y))
        print("MaybeErrrorWhenConvertingPolygonToRectangle")
        #raise Exception
        #matrixCrop = (0,0,0,0)
    return matrixCrop
def findModeLongWay(lst):
    if len(lst) == 0:
        return 0
    from collections import Counter
    d_mem_count = Counter(lst)
    modeLst = []
    for k in d_mem_count.keys():
        if d_mem_count[k] > 1:
            modeLst.append(k)
    try:
        return int(round(statistics.mean(modeLst)))
    except:
        return 0
def cutTopandBottomBlackRowsFunction(new_img):
    # GO THROUGH EACH ROW IN ORGINAL PICTURE AND MARK OUT THE ROWS THAT NEED TO BE DELETED
    deleteRows = []
    rowNum = 0
    for row in new_img:
        ct = 0
        for rgb in row:
            if not all(rgb == 0):
                break
            else:
                ct+=1
        #print(ct==len(row))
        if (ct==len(row)):
            deleteRows.append(rowNum)
        rowNum+=1
    # GO THROUGH EACH ROW AND DELETE THE ROWS -- IMG CROPPED TOP AND BOTTOM
    rowCt = 0
    croppedTopBottomImg = []
    for row in new_img:
        if rowCt not in deleteRows:
            newrow = []
            for rgb in row:
                #print(rgb)
                newrow.append([rgb[0], rgb[1], rgb[2]])
            #print(newrow)
            #print(row)
            #print()
            croppedTopBottomImg.append(newrow)
        #print(rowCt)
        rowCt+=1
    return croppedTopBottomImg
def convertImgto01OrginalFunction(new_img):
    # CONVERT IMAGE TO 0 AND 1 IMAGE FROM THE ORGINAL IMAGE
    newImgOnly01 = []
    for row in new_img:
        newRow = []
        for rgb in row:
            newArrayRowz = []
            for color in rgb:
                if color >= 0.5:
                    newArrayRowz.append(255)
                else:
                    newArrayRowz.append(0)
            newRow.append(newArrayRowz)
        newImgOnly01.append(newRow)
    return newImgOnly01
def cropTopBottomFrom01Img(newImgOnly01):
    # FIND ROWS THAT NEED TO BE DELETED FROM 0 AND 1 IMAGE
    deleteRows = []
    rowNum = 0
    for row in newImgOnly01:
        ct = 0
        #print(row)
        for rgb in row:
            #print(rgb)
            if not rgb == [0,0,0]:
                break
            else:
                ct+=1
        #print(ct==len(row))
        if (ct==len(row)):
            deleteRows.append(rowNum)
        rowNum+=1
    # CREATE NEW IMAGE WITH CROPPED 0 AND 1 IMAGES
    rowCt = 0
    croppedTopBottomImgOnly01 = []
    for row in newImgOnly01:
        if rowCt not in deleteRows:
            croppedTopBottomImgOnly01.append(row)
        #print(rowCt)
        rowCt+=1
    return croppedTopBottomImgOnly01
def percentAreafromImg(img):
    if len(img) == 0:
        return 0
    ctYes = 0
    for row in img:
        for value in row:
            if value != [0,0,0]:
                ctYes+=1
    return round(ctYes/(len(croppedTopBottomImgOnly01)*len(croppedTopBottomImgOnly01[0]))*100,4)
def displayImg(arrayofImgs):
    plt.figure()
    f, axarr = plt.subplots(1,len(display)) 
    for i in range(len(display)):
        axarr[i].imshow(display[i])
def findPosition(value):
    rowNumb = 0
    for i in dfListNew:
        if value == i[0]:
            return (rowNumb)
        rowNumb+=1
    return 0
def findInfoSingle(i):
    if i[1] == True and i[2] == True and i[0].lower() in vowelsNotI:
        return True
    elif i[1] == True and i[2] == True:
        return True
    else:
        return False
def findInfo(wordInfo):
    totalPossibleWeight = 0
    ctTrue1 = 0
    ctTrue2 = 0
    weight = 0
    for i in wordInfo:
        totalPossibleWeight+=4 if i in vowelsNotI else 3
        if i[1] == True:
            ctTrue1+=1
        if i[2] == True:
            ctTrue2+=1
        if not i[0].isalpha():
            weight+=-1
        elif i[1] == True and i[2] == True and i[0].lower() in vowelsNotI:
            weight+=5
        elif i[1] == True and i[2] == True and not i[0].lower() in vowelsNotI:
            weight+=3    
        elif (i[1] == True or i[2] == True) and i[0].lower() in vowelsNotI:
            weight+=2
        elif (i[1] == True or i[2] == True) and not i[0].lower() in vowelsNotI:
            weight+=1
        elif i[1] == False or i[2] == False and i[0].lower() in vowelsNotI:
            weight+=-1
        elif i[1] == False or i[2] == False and not i[0].lower() in vowelsNotI:
            weight+=0
        else:
            print("ERROR")
    #print("Weighted Score " + str((weight/totalPossibleWeight)*100))
    #print("Total Percent Bold " + str(((ctTrue1+ctTrue2)/(len(wordInfo)*2))*100))
    #print("Test 1 only Percent Bold " + str(((ctTrue1)/(len(wordInfo)))*100))
    #print("Test 2 only Percent Bold " + str(((ctTrue2)/(len(wordInfo)))*100))
    if (weight/totalPossibleWeight)*100 < 0:
        weightscore = 0
    else:
        weightscore = (weight/totalPossibleWeight)*100
    return [weightscore, ((ctTrue1+ctTrue2)/(len(wordInfo)*2))*100, ((ctTrue1)/(len(wordInfo)))*100, ((ctTrue2)/(len(wordInfo)))*100]
def findMatrix(vert):
    matrix = ((vert.vertices[0].x, vert.vertices[0].y),
              (vert.vertices[1].x, vert.vertices[1].y),
              (vert.vertices[2].x, vert.vertices[2].y),
              (vert.vertices[3].x, vert.vertices[3].y))
    return matrix
def strokewidthofChar(lstValue, quartile):
    #print(0) #0th percentile
    #print(int(heightPic/4)+1) #25 percentile
    #print(int((heightPic/4)*2)+1) #50 percentile
    #print(int((heightPic/4)*3)+1) #75 percentile
    #print(heightPic)#100 percentile
    lstStokeWidth = []
    for i in lstValue:
        running = i
        heightPic = len(running)
        rowNum = 1
        if quartile == "0-25":
            start = 0
            end = int(heightPic/4)+1
        elif quartile == "25-75":
            start = int(heightPic/4)+1
            end = int((heightPic/4)*3)+1
        elif quartile == "75-100":
            start = int((heightPic/4)*3)+1
            end = heightPic
        else:
            print("invalid input for range")
            raise Exception
        for row in running:
            if rowNum < end and rowNum >= start:
                ctYes = 0
                for i in row:
                    if i == [255,255,255]:
                        ctYes+=1
                lstStokeWidth.append(ctYes)
            rowNum+=1
    return lstStokeWidth
def findq1q3(datasetInput):
    dataset = sorted(datasetInput)
    q1, q3= np.percentile(dataset,[25,75])
    return q1, q3
def reject_outliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]


In [38]:
def RUNALL(imagePath):
    keyDIR = imgTxtVisionAPIPath #JSON key file to call the api
    credentials = service_account.Credentials.from_service_account_file(keyDIR) #using service account to go through google
    client = vision.ImageAnnotatorClient(credentials=credentials) # client api
    bounds = []
    with io.open(imagePath, 'rb') as image_file:
        content = image_file.read()
    image = types.Image(content=content)
    response = client.document_text_detection(image=image)
    global document
    document = response.full_text_annotation
    alphaList = list(string.ascii_lowercase) + list(string.ascii_uppercase)
    allbbChar = []
    for charater in alphaList:
        charbb = []
        for page in document.pages:
            for block in page.blocks:
                for paragraph in block.paragraphs:
                    for word in paragraph.words:
                        for symbol in word.symbols:
                            if symbol.text == charater:
                                charbb.append(symbol.bounding_box)
        allbbChar.append(charbb)
    dfList = []
    for i in range(len(allbbChar)):
        charLen = [alphaList[i], len(allbbChar[i])]
        dfList.append(charLen)
        #print(alphaList[i] + "\t" +  str(len(allbbChar[i])))
    df=pd.DataFrame(dfList, columns=['Char','NumberOfChars']) 
    totalWHAllChar = []
    for i in range(len(allbbChar)):
        heightChar = []
        widthChar = []
        for singleChar in allbbChar[i]:
            cropPoints = convertBoundBoxtodiagonalRectangual(singleChar)
            im = Image.open(imagePath).convert("RGBA")
            im_crop = im.crop(cropPoints)
            opencvImage = cv2.cvtColor(np.array(im_crop), cv2.COLOR_RGB2BGR)
            img_reverted= cv2.bitwise_not(opencvImage)
            new_img = img_reverted / 255.0 
            heightChar.append(len(new_img))
            widthChar.append(len(new_img[0]))
        #totalWHAllChar.append([heightChar, widthChar])
        try:
            totalWHAllChar.append([statistics.mode(heightChar), statistics.mode(widthChar)])
        except:
            totalWHAllChar.append([findModeLongWay(heightChar), findModeLongWay(widthChar)])
    dfListNew = []
    #totalWHAllChar
    for i in range(len(dfList)):
        dfListNew.append([dfList[i][0], dfList[i][1], totalWHAllChar[i][0], totalWHAllChar[i][1]])
    df=pd.DataFrame(dfListNew, columns=['Char','NumberOfChars','ModeHeight', 'ModeWidth']) 
    templst = []
    strokeWidthArray = []
    for i in range(len(allbbChar)):
        areaAllForChar = []
        pictureForChar = []
        for singleChar in allbbChar[i]:
            cropPoints = convertBoundBoxtodiagonalRectangual(singleChar)
            im = Image.open(imagePath).convert("RGBA")
            im_crop = im.crop(cropPoints)
            opencvImage = cv2.cvtColor(np.array(im_crop), cv2.COLOR_RGB2BGR)
            img_reverted= cv2.bitwise_not(opencvImage)
            new_img = img_reverted / 255.0 
            #croppedTopBottomImg = cutTopandBottomBlackRowsFunction(new_img)
            newImgOnly01 = convertImgto01OrginalFunction(new_img)
            croppedTopBottomImgOnly01 = cropTopBottomFrom01Img(newImgOnly01)
            #display = [new_img,croppedTopBottomImg, newImgOnly01, croppedTopBottomImgOnly01]
            #displayImg(display)
            percentAreaConverd = percentAreafromImg(croppedTopBottomImgOnly01)
            areaAllForChar.append(percentAreaConverd)
            pictureForChar.append(croppedTopBottomImgOnly01)
        templst.append(areaAllForChar)
        strokeWidthArray.append(pictureForChar)
    for i in range(len(dfListNew)):
        if not (dfListNew[i][1] == len(templst[i])):
            raise ValueError('The values are not equal')
    def findOutlierCutoffIQR(datasetInput):
        dataset = sorted(datasetInput)
        q1, q3= np.percentile(dataset,[25,75])
        iqr = q3 - q1
        return q3 +(1.5 * iqr) 
    dfListNew2 = []
    for i in range(len(dfListNew)):
        charLstbefore = [dfListNew[i][0], dfListNew[i][1], dfListNew[i][2], dfListNew[i][3]]
        if not len(templst[i]) == 0:
            charLstbefore.append(round(statistics.median(templst[i]),4))
            charLstbefore.append(round(statistics.mean(templst[i]),4))
            try:
                charLstbefore.append(round(statistics.mode(templst[i]),4))
            except:
                charLstbefore.append(round(statistics.median(templst[i]),4))
            charLstbefore.append(round(findOutlierCutoffIQR(templst[i]),4))
            charLstbefore.append(round(len(detect_Maximum_outlier(templst[i])),4))
            charLstbefore.append(round(len(findMaxOutliersIQR(templst[i])),4))

        else:
            charLstbefore.extend([0,0,0,0,0,0])
        dfListNew2.append(charLstbefore)
    colName = ['Char','NumberOfChars','ModeHeight','ModeWidth','MedianArea','MeanArea','ModeArea', 'MaxOutlierNum', 'NumOutlierZScore', 'NumOutlierIQR']
    df=pd.DataFrame(dfListNew2, columns=colName) 
    FINALALLINFOLIST = dfListNew2
    print("There is " + str(len(strokeWidthArray)) + " symbols in the array.\nIts a,b,c,d...x,y,z,A,B,C...X,Y,Z.\nIn each Position is the RGB for that image")
    print("Example")
    print(strokeWidthArray[0][0][:2])
    for i in dfListNew:
        if i[1] < 10:
            print("Char "+ i[0] + " has only " + colored(str(i[1]), 'red', attrs=['bold']))
    positions = [["b","0-25"],
                 ["c","25-75"],
                 ["d","0-25"],
                 ["f","75-100"],
                 ["h","0-25"],
                 ["p","75-100"],
                 ["q","75-100"],
                 ["t","75-100"],
                 ["F","75-100"],
                 ["L","0-25"],
                 ["T","75-100"],
                 ["Y","75-100"]]
    newvalue = []
    for i in positions:
        x = i
        x.append(findPosition(i[0]))
        newvalue.append(x)
    print(newvalue)
    totalStokeWidth = []
    for i in newvalue:
        totalStokeWidth.extend(strokewidthofChar(strokeWidthArray[i[2]], i[1]))
    q1, q3 = findq1q3(totalStokeWidth)
    print("All Values")
    lstofCount = []
    for j in list(set(totalStokeWidth)):
        lstofCount.append([j, sum(i == j for i in totalStokeWidth)])
        #print(str(j) + "\t" + str(sum(i == j for i in totalStokeWidth)))
    dfForStrokeWidth=pd.DataFrame(lstofCount, columns=['Detected Width','NumberOfCharsDetected']) 
    print("Stroke width is between " + str(q1) + " - " + str(q3) + ".\nMost likely --> " + str(round(statistics.mean(totalStokeWidth),4)))
    STROKEWIDTHFINAL = round(statistics.mean(totalStokeWidth),4)
    FINALDFALLCHARINFOLST = []
    for char in FINALALLINFOLIST:
        tempArr = char
        tempArr.extend([STROKEWIDTHFINAL])
        FINALDFALLCHARINFOLST.append(tempArr)
    colName = ['Char','NumberOfChars','ModeHeight','ModeWidth','MedianArea','MeanArea','ModeArea', 'MaxOutlierNum', 'NumOutlierZScore', 'NumOutlierIQR', 'AvgStrokeWidth']
    df=pd.DataFrame(FINALDFALLCHARINFOLST, columns=colName) 
    os.chdir("/Users/kunal/Documents/VdartResumeProject/Erosion/")
    writer = ExcelWriter(os.path.basename(imagePath)[:-4] + '.xlsx')
    df.to_excel(writer,'Sheet1')
    writer.save()
    lsttester = []
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    runningWord = ""
                    uppercaseLetterCt = 0
                    totalSymbolsLeterCt = 0
                    for symbol in word.symbols:
                        if symbol.text.isupper():
                            uppercaseLetterCt+=1
                        totalSymbolsLeterCt+=1
                        runningWord+=symbol.text
                    if uppercaseLetterCt == totalSymbolsLeterCt:
                        lsttester.append(findArea(word.bounding_box))
    try:
        thresholdAreaCapitailWord = min(findMaxOutliersIQR(lsttester))                 
    except Exception as ValueError:
        thresholdAreaCapitailWord = max(lsttester)    
    boundingBoxForLargeCapital = []
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    runningWord = ""
                    uppercaseLetterCt = 0
                    totalSymbolsLeterCt = 0
                    for symbol in word.symbols:
                        if symbol.text.isupper():
                            uppercaseLetterCt+=1
                        totalSymbolsLeterCt+=1
                        runningWord+=symbol.text
                    if uppercaseLetterCt == totalSymbolsLeterCt and findArea(word.bounding_box) > thresholdAreaCapitailWord:
                        #print(runningWord)
                        #print(findArea(word.bounding_box))
                        boundingBoxForLargeCapital.append(word.bounding_box)
    boxesForCharsGoodMean = []
    boxesForCharsbadMean = []
    boxesForCharsGoodOutlier = []
    boxesForCharsbadOutlier = []
    smtg = []
    for page in document.pages:
        pg = []
        for block in page.blocks:
            blk = []
            for paragraph in block.paragraphs:
                para = []
                for word in paragraph.words:
                    numSymbols = 0
                    wordText = ""
                    wordInfo = []
                    for symbol in word.symbols:
                        symbolInfo = []
                        wordText += symbol.text
                        cropPoints = convertBoundBoxtodiagonalRectangual(symbol.bounding_box)
                        im = Image.open(imagePath).convert("RGBA")
                        im_crop = im.crop(cropPoints)
                        opencvImage = cv2.cvtColor(np.array(im_crop), cv2.COLOR_RGB2BGR)
                        img_reverted= cv2.bitwise_not(opencvImage)
                        new_img = img_reverted / 255.0 
                        #croppedTopBottomImg = cutTopandBottomBlackRowsFunction(new_img)
                        newImgOnly01 = convertImgto01OrginalFunction(new_img)
                        croppedTopBottomImgOnly01 = cropTopBottomFrom01Img(newImgOnly01)
                        #display = [new_img,croppedTopBottomImg, newImgOnly01, croppedTopBottomImgOnly01]
                        #displayImg(display)
                        percentAreaConverd = percentAreafromImg(croppedTopBottomImgOnly01)
                        boxUpload = False
                        currentSymbolMean = False
                        currentSymbolOutlier = False
                        for charGroup in dfListNew2:
                            if charGroup[0] == symbol.text.strip().lower(): 
                                boxUpload = True
                                #----------------------------------------------------
                                boxesForCharsGoodMean.append(symbol.bounding_box) if percentAreaConverd >= charGroup[5] else boxesForCharsbadMean.append(symbol.bounding_box)
                                currentSymbolMean = True if percentAreaConverd >= charGroup[5] else False
                                #----------------------------------------------------
                                boxesForCharsGoodOutlier.append(symbol.bounding_box) if percentAreaConverd >= charGroup[7] else boxesForCharsbadOutlier.append(symbol.bounding_box) 
                                currentSymbolOutlier = True if percentAreaConverd >= charGroup[7] else False

                        if not boxUpload:
                            #print("Running Extra -->" + symbol.text)
                            #----------------------------------------------------
                            boxesForCharsGoodMean.append(symbol.bounding_box) if percentAreaConverd >= 40 else boxesForCharsbadMean.append(symbol.bounding_box)
                            currentSymbolMean = True if percentAreaConverd >= 40 else False
                            #----------------------------------------------------
                            boxesForCharsGoodOutlier.append(symbol.bounding_box) if percentAreaConverd >= 40 else boxesForCharsbadOutlier.append(symbol.bounding_box) 
                            currentSymbolOutlier = True if percentAreaConverd >= 40 else False


                        symbolInfo.extend([symbol.text, currentSymbolMean, currentSymbolOutlier, symbol.bounding_box])
                        wordInfo.append(symbolInfo)
                    wordInfo.append(wordText)
                    wordInfo.append(word.bounding_box)
                    para.append(wordInfo)
                    #para.append()
                blk.append(para)
                blk.append(paragraph.bounding_box)
            pg.append(blk)
            pg.append(block.bounding_box)
        smtg.append(pg)
    print("Finished")
    smtg = smtg[0]
    vowelsNotI = ['a','e','o','u']
    BD = []
    for i in range(0, len(smtg), 2):
        #print("BLOCK " + str(i/2) + "\t\t" + str(findMatrix(smtg[i+1])))
        tttt=smtg[i]
        #print(str(len(tttt)/2) + " Paragraphs")
        for i in range(0, len(tttt), 2):
            #print("\t" + str(i/2) + "\tBounding Box Para\t" + str(findMatrix(tttt[i+1])))
            newloop = tttt[i]
            #print("\t\t" + str(len(newloop)) + "\t Words")
            stringPrint = ""
            totalScoreForPara = []
            for r in newloop:
                #print(rrrr)
                wordArray = r[:-2]
                removedBDWordArray = []
                for t in wordArray:
                    removedBDWordArray.append(t[:-1])
                #print(wordArray[-1])
                #for symbol in wordArray:
                    #print("Letter " + symbol[0] + " Test1 " + str(symbol[1]) + " Test2 " + str(symbol[2]))
                scoreAll = findInfo(removedBDWordArray)
                #print("fds" + str(scoreAll))

                if statistics.mean(scoreAll) > 80:
                    BD.append(r[-1])


                #print(r[-2]  + " "+ str(scoreAll))
                totalScoreForPara.append(scoreAll[0])
                stringPrint += r[-2] + " "
                #print(r[-1])
            if statistics.mean(totalScoreForPara) > 40:
                BD.append(tttt[i+1])
                #print("Yes")
            #else:
                #Uncomment if you want it to also highlight small words that are bolded
                #for r in newloop:
                    #wordArray = r[:-1]
                    #for symbol in wordArray:
                        #print(symbol)
                        #if findInfoSingle(symbol) and symbol[0].isalpha(): 
                            #BD.append(symbol[-1])                    
            #print("\t\t" + stringPrint)
            #print("-----------------------------------------------------------")
        #print("===========================================================")
    joinedLst = boundingBoxForLargeCapital + BD
    pointsforBD = []
    for i in joinedLst:
        pointsforBD.append(findMatrix(i))
    ypointsLine = []
    for i in pointsforBD:
        ypointsLine.append(i[0][1])
    ypointsLine.sort()
    for x in ypointsLine:
        print(x, end=' ')
    TINT_COLOR = (0,0,0) 
    #colors = [(0,255,0), (0,0,255), (255,0,0)]
    TRANSPARENCY = 0.25
    OPACITY = int(255 * TRANSPARENCY)
    img = Image.open(imagePath)
    img = img.convert("RGBA")
    overlay = Image.new('RGBA', img.size, TINT_COLOR+(0,))
    draw = ImageDraw.Draw(overlay)  # Create a context for drawing things on it.
    x, y= img.size
    TINT_COLOR = (0,255,0) 
    for para in BD:
        matrix = findMatrix(para)
        draw.polygon(matrix, fill=TINT_COLOR+(OPACITY,))
    TINT_COLOR = (255,0,0) 
    for para in boundingBoxForLargeCapital:
        matrix = findMatrix(para)
        draw.polygon(matrix, fill=TINT_COLOR+(OPACITY,))
    for yPos in ypointsLine:
        shape = [(0, yPos-5), (x, yPos-5)]
        draw.line(shape, fill ="black", width = 1) 
    img = Image.alpha_composite(img, overlay)
    img = img.convert("RGB") # Remove alpha for saving in jpg format.
    os.chdir("/Users/kunal/Documents/VdartResumeProject/VisionAPi/ErosionSaveImg")
    img.save(os.path.basename(imagePath)[:-4] + ".jpg")
    return img